<a href="https://colab.research.google.com/github/shinchacoffee/2_circles_eparation_simple/blob/main/pytorch_tutorial_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
random_seed = 1
torch.manual_seed(random_seed)  

In [ ]:
training_data = datasets.FashionMNIST(root='data', 
                                      train=True, 
                                      download=True, 
                                      transform=ToTensor()
                                      )

test_data = datasets.FashionMNIST(root='data',
                                  train=False,
                                  download=True,
                                  transform=ToTensor()
                                  )

In [ ]:
batch_size = 32

train_data_loader = DataLoader(training_data, batch_size=batch_size)
test_data_loader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_data_loader:
  print(f"Shape of features (X) [N, C, H, W]: {X.shape}")
  print(f"Shape of labels (y): {y.shape} {y.dtype}")
  #print("Shape of labels (y): {} {}".format(y.shape, y.dtype))
  #print("Shape of labels (y):",y.shape,y.dtype)
  break


Shape of features (X) [N, C, H, W]: torch.Size([32, 1, 28, 28])
Shape of labels (y): torch.Size([32]) torch.int64


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"using {device} device")

using cuda device


In [ ]:
from random import shuffle
a = [1,2,3]
a = shuffle(a)
print(a)

None


In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(512),
        nn.Linear(512,512),
        nn.ReLU(512),
        nn.Linear(512,10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
for m in model.parameters():
  print(m.grad)
  break

None


In [ ]:
def train(dataloader, model, lossfn, optimizer, gradaccu=1):
  size = len(dataloader.dataset)
  model.train()
  optimizer.zero_grad()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    
    #if batch % gradaccu == 0:
      #optimizer.zero_grad()
      #print(f"zerujem for batch: {batch}")
    
    pred = model(X)
    loss = lossfn(pred, y) / gradaccu
    #for m in model.parameters():
    #  print(m.shape, m.grad)
    #  break
    loss.backward()
      #for m in model.parameters():
      #  print(m.shape, m.grad[11,11])
      #  break
    #pred1 = model(X)
    #loss1 = lossfn(pred1, y)
    #loss1.backward()
    #for m in model.parameters():
    #  print(m.shape, m.grad[11,11])
    #  break
    #print(model.linear_relu_stack[2].weight[0,0].grad)

    if batch % gradaccu == (gradaccu-1):
      optimizer.step()
      optimizer.zero_grad()
    else:
      if (batch+1) *  batch_size > size:
        optimizer.step()
        optimizer.zero_grad()
      #print(f"optimizer step for batch: {batch}")

      #if batch == 10:
      #  break

    if batch % 400 == 0:
      loss, current = loss.item(), batch*len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
print(model.linear_relu_stack[0].weight[0,0].grad)

None


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten/src/ATen/core/TensorBody.h:417.)
  return self._grad


In [ ]:
aa = torch.ones(3, requires_grad=True)
bb = torch.ones(3, requires_grad=True)
cc = aa + bb
cc = cc.sum()

In [ ]:
print(aa.grad)
aa.grad = None
cc.backward()
print(aa.grad)

None
tensor([1., 1., 1.])


In [ ]:
def test(dataloader, model, lossfn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += lossfn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"test error: \n accuracy: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f} \n")

In [ ]:
#model.load_state_dict(torch.load("model.pth"))
epochs = 4
for t in range(epochs):
  print(f"epoch {t+1}\n------------------------------")
  model.to(device)
  train(train_data_loader, model, loss_fn, optimizer, gradaccu=1)
  test(test_data_loader, model, loss_fn)
print("Done! lul :x")

epoch 1
------------------------------
loss: 2.294393 [    0/60000]
loss: 2.234691 [12800/60000]
loss: 2.192353 [25600/60000]
loss: 2.154234 [38400/60000]
loss: 2.049330 [51200/60000]
test error: 
 accuracy: 54.0%, avg loss: 1.869573 

epoch 2
------------------------------
loss: 1.850787 [    0/60000]
loss: 1.692901 [12800/60000]
loss: 1.564777 [25600/60000]
loss: 1.531326 [38400/60000]
loss: 1.364210 [51200/60000]
test error: 
 accuracy: 63.6%, avg loss: 1.236459 

epoch 3
------------------------------
loss: 1.217542 [    0/60000]
loss: 1.117362 [12800/60000]
loss: 1.073575 [25600/60000]
loss: 1.222296 [38400/60000]
loss: 1.025919 [51200/60000]
test error: 
 accuracy: 66.4%, avg loss: 0.973263 

epoch 4
------------------------------
loss: 0.909452 [    0/60000]
loss: 0.866163 [12800/60000]
loss: 0.869220 [25600/60000]
loss: 1.112815 [38400/60000]
loss: 0.877733 [51200/60000]
test error: 
 accuracy: 69.1%, avg loss: 0.852328 

Done! lul :x


In [ ]:
torch.save(model.state_dict(), "model.pth")
print("saved pytorch model state to model.pth")

saved pytorch model state to model.pth


In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model2 = NeuralNetwork()
model2.load_state_dict(torch.load("model.pth"))

model.to("cpu")
model.eval()
x, y = test_data[1][0], test_data[1][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Pullover", Actual: "Pullover"
